## Intro

In [1]:
import pandas as pd
import shared_utils
import matplotlib as plt
#set_option to increase max rows displayed to 200, to see entire df in 1 go/
pd.set_option("display.max_rows", 200)

In [10]:
#find out how to import cleaning scrip from other TIRCP notebook

from ..tircp import A1_data_prep

ImportError: attempted relative import with no known parent package

## Read in Raw data

In [2]:
url='gs://calitp-analytics-data/data-analyses/bus_procurement_cost/TIRCP Tracking Sheets 2_1-10-2024.xlsx'
sheet_name='Agreement Allocations'


In [3]:
df = pd.read_excel(url,sheet_name)

In [4]:
display(
    df.shape,
    type(df),
    df.columns
)

(726, 55)

pandas.core.frame.DataFrame

Index(['Award Year', 'Project #', 'Grant Recipient', 'Implementing Agency',
       'PPNO', 'Project ID', 'EA', 'Components', '#of buses', 'Phase',
       'Allocation Amount', 'Expended Amount', 'SB1 Funding',
       'SB1 Budget Year', 'GGRF Funding', 'GGRF Budget Year', 'GF Funding',
       'GF Budget Year', 'CTC Financial Resolution',
       'CTC Allocation Amendment', 'CTC Waiver', 'CalSTA Waiver',
       'Allocation Date', 'Phase Completion Date (SAR)', 'PSA #',
       'CT Document #', '3rd Party Award Date', 'LED', 'Termination Date',
       'Final Invoice Date', 'Date Branch Chief Receives PSA',
       'Date Regional Coordinator Receives PSA', 'Date OC Receives PSA',
       'Date OPM Receives PSA', 'Date Legal Receives PSA',
       'Date Returned to PM', 'Date PSA Sent to Local Agency',
       'Date PSA Approved by Local Agency', 'Date Signed by DRMT',
       'PSA Expiry Date', 'LONP', 'Prior Fiscal Years to 2020',
       'Fiscal Year 2020-2021', 'Fiscal Year 2021-2022',
       'F

## Data Cleaning and QC

In [10]:
#reducing initialdf to first 11 columns.
tircp =df.iloc[:, :12]

In [19]:
display(
    tircp.shape,
    list(tircp.columns))

(726, 12)

['Award Year',
 'Project #',
 'Grant Recipient',
 'Implementing Agency',
 'PPNO',
 'Project ID',
 'EA',
 'Components',
 '#of buses',
 'Phase',
 'Allocation Amount',
 'Expended Amount']

In [20]:
#dictionary for column name update
ew_col=[
    'award_year',
    'project_#',
    'grant_recipient',
    'implementing_agency',
    'ppno',
    'project_id',
    'ea',
    'components',
    '#_of_buses',
    'phase',
    'allocation_amount',
    'expended_amount']

In [21]:
tircp.columns = new_col
tircp.columns

Index(['award_year', 'project_#', 'grant_recipient', 'implementing_agency',
       'ppno', 'project_id', 'ea', 'components', '#_of_buses', 'phase',
       'allocation_amount', 'expended_amount'],
      dtype='object')

In [27]:
#data type checking
tircp.dtypes
#everything looks good. can drop expanded_amount

award_year             float64
project_#              float64
grant_recipient         object
implementing_agency     object
ppno                    object
project_id             float64
ea                      object
components              object
#_of_buses             float64
phase                   object
allocation_amount        int64
expended_amount         object
dtype: object

In [29]:
tircp = tircp.drop('expended_amount', axis=1)

In [30]:
tircp.head()

,award_year,project_#,grant_recipient,implementing_agency,ppno,project_id,ea,components,#_of_buses,phase,allocation_amount
0,2018.0,28.0,Transportation Agency for Monterey County,Transportation Agency for Monterey County,1155,NaN,NaN,New Passenger Rail Service to Salinas,NaN,CONST,0
1,2018.0,28.0,Transportation Agency for Monterey County,Transportation Agency for Monterey County,1155,NaN,NaN,Positive Train Control,NaN,CONST,0
2,2016.0,11.0,San Bernardino County Transportation Authority...,San Bernardino County Transportation Authority...,1230,19000086.0,R391GD,Construct New Rail System Linking San Bernardi...,NaN,CONST,9204000
3,2020.0,10.0,San Bernardino County Transportation Authority...,San Bernardino County Transportation Authority,1232,23000073.0,T458GA,Acquisition of Zero-Emission Buses,NaN,CONST,15000000
4,2018.0,19.0,San Joaquin Joint Powers Authority and San Joa...,San Joaquin Regional Rail Commission,10-9883,NaN,NaN,Stockton Diamond,NaN,CONST,0


In [31]:
# fill NaN with zero?
# see if you can sum the bus column
tircp.agg({'#_of_buses':'sum'})
# nope this is correct

#_of_buses    527.0
dtype: float64

## Export Cleaned data

In [32]:
tircp.to_csv('gs://calitp-analytics-data/data-analyses/bus_procurement_cost/tircp_allocations_bus_only_clean.csv')

## Read in Cleaned data from GCS

## Initial Summary Stats

In [ ]:
import re

input_string = "Renovation and expansion of the Monterey maintenance and operations facility, Reduced Deadheads for 30 buses, increased transit service connecting East Salinas and the Salinas Intermodal Station. procure 114 new buses"

# Extract all numbers from the string
numbers = re.findall(r'\d+', input_string)

print("Extracted Numbers:", numbers)
